In [4]:
from impaintingLib import *

trainloader, testloader = getFaces()

model     = AutoEncoder().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.001)
criterion = torch.nn.L1Loss()

### Visualization :
# - plot_all_img 
# - plot_last_img
# - board_plot_last

# Train
alterFunc = Alter(min_cut=4, max_cut=60).squareMask
visuFunc = None
train(model, optimizer, trainloader, criterion, epochs=1, alter=alterFunc, visu=visuFunc)

# Test
visuFunc = Visu(runName = "AutoEncodeEpo1",save=False).board_plot_last
test(model, testloader, alter=alterFunc, visu=visuFunc)

# images, labels = next(iter(trainloader))
# images = alterFunc(images)
# visuFunc(images)

  0%|          | 0/94 [00:00<?, ?it/s]